In [1]:
!pip install transformers
#!pip install iterative-stratification

In [2]:
!git clone https://github.com/sronger/PSG_Predicting_Algorithm_Tags_and_Difficulty.git

fatal: destination path 'PSG_Predicting_Algorithm_Tags_and_Difficulty' already exists and is not an empty directory.


In [3]:
%cd PSG_Predicting_Algorithm_Tags_and_Difficulty

/Users/klarakjomefischer/Downloads/Carleton/PSG_Predicting_Algorithm_Tags_and_Difficulty


In [4]:
import pandas as pd

import random
import numpy as np

from transformers import AutoConfig, AutoModel, AutoTokenizer, RobertaTokenizer
#from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from torch.utils.data import TensorDataset, DataLoader

from tqdm import tqdm
from datetime import datetime
from collections import Counter, defaultdict
import os
import shutil
from itertools import chain

import ast

In [5]:
print(os.listdir('..'))

['.DS_Store', 'PSG_Predicting_Algorithm_Tags_and_Difficulty', 'Copy_of_BigBird_based_Single_Task_Model.ipynb', '.venv', 'NLP', '.ipynb_checkpoints']


In [6]:
#from google.colab import drive
import pandas as pd

# Mount Google Drive
#drive.mount('/content/drive')
# Define the paths to the JSON files
train_file_path = '../NLP/FinalProject/codeforces_train.jsonl'
test_file_path = '../NLP/FinalProject/codeforces_test.jsonl'

# Load the JSON files into pandas DataFrames
try:
    train_df = pd.read_json(train_file_path, lines=True)
    test_df = pd.read_json(test_file_path, lines=True)
    print("Datasets loaded successfully!")
    print("Train dataset head:")
    display(train_df.head()) # Display the first few rows of the train DataFrame
    print("\nTest dataset head:")
    display(test_df.head()) # Display the first few rows of the test DataFrame
except FileNotFoundError:
    print(f"Error: One of the files was not found. Please check the paths.")
except Exception as e:
    print(f"An error occurred while loading the files: {e}")

Datasets loaded successfully!
Train dataset head:


,statement,greedy,math,implementation,dp,constructive algorithms,data structures,brute force,binary search,sortings,graphs
0,After years of hard work scientists invented a...,1,0,0,0,1,0,0,0,0,0
1,Traveling around the world you noticed that ma...,0,0,0,1,0,0,0,0,0,0
2,"Burenka is the crown princess of Buryatia, and...",1,1,0,1,0,0,0,0,0,0
3,Once Bob needed to find the second order stati...,0,0,0,0,0,0,1,0,0,0
4,Petya and Gena love playing table tennis. A si...,0,0,0,0,0,0,0,1,0,0



Test dataset head:


,statement,greedy,math,implementation,dp,constructive algorithms,data structures,brute force,binary search,sortings,graphs
0,The black king is standing on a chess field co...,0,0,0,0,0,0,0,0,0,1
1,"You are given an integer x of n digits a_1, a_...",1,0,1,0,1,0,0,0,0,0
2,"You are organizing a boxing tournament, where ...",1,0,0,1,0,0,1,0,0,0
3,Sherlock met Moriarty for a final battle of wi...,0,0,1,0,1,1,0,0,0,1
4,Of course you have heard the famous task about...,1,0,0,1,0,1,1,0,1,0


In [7]:
old_train_df = pd.read_csv('./data/AMT10/AMT10_train.csv', index_col=0, encoding='utf8')
old_valid_df = pd.read_csv('./data/AMT10/AMT10_validation.csv', index_col=0, encoding='utf8')
old_train_df.head()

,description,tags,rating
1845/E,$$$ n $$$ box place a line box number $$$ 1 $$...,"['dp', 'implementation', 'math']",2500.0
1845/B,bob carol hang alice whole day 's time go home...,"['geometry', 'implementation', 'math']",900.0
1845/A,give integer $$$ n $$$ want obtain unlimited s...,"['constructive algorithms', 'implementation', ...",800.0
1841/B,array $$$ [ a_1 a_2 \dots a_k ] $$$ call beaut...,['implementation'],1000.0
1839/A,give two integers $$$ n $$$ $$$ k $$$ array $$...,"['greedy', 'implementation', 'math']",800.0


In [8]:
# Rename 'statement' to 'description' for consistency with old_train_df
train_df = train_df.rename(columns={'statement': 'description'})
test_df = test_df.rename(columns={'statement': 'description'})

# Identify all boolean tag columns (all columns except 'description')
jsonl_tag_columns = [col for col in train_df.columns if col != 'description']

# Create a new 'tags' column as a list of strings
# Each string is the name of a tag column where the value is 1
train_df['tags'] = train_df.apply(lambda row: [col for col in jsonl_tag_columns if row[col] == 1], axis=1)
test_df['tags'] = test_df.apply(lambda row: [col for col in jsonl_tag_columns if row[col] == 1], axis=1)

# Drop the individual boolean tag columns as they are now consolidated into the 'tags' list
train_df = train_df.drop(columns=jsonl_tag_columns)
test_df = test_df.drop(columns=jsonl_tag_columns)

# Note: Your JSONL data does not contain a 'rating' column, unlike 'old_train_df'.
# If you intend to train for the 'rating' task, you will need to add a 'rating' column with actual data.

print("New train_df head:")
display(train_df.head())
print("\nNew test_df head:")
display(test_df.head())

New train_df head:


,description,tags
0,After years of hard work scientists invented a...,"[greedy, constructive algorithms]"
1,Traveling around the world you noticed that ma...,[dp]
2,"Burenka is the crown princess of Buryatia, and...","[greedy, math, dp]"
3,Once Bob needed to find the second order stati...,[brute force]
4,Petya and Gena love playing table tennis. A si...,[binary search]



New test_df head:


,description,tags
0,The black king is standing on a chess field co...,[graphs]
1,"You are given an integer x of n digits a_1, a_...","[greedy, implementation, constructive algorithms]"
2,"You are organizing a boxing tournament, where ...","[greedy, dp, brute force]"
3,Sherlock met Moriarty for a final battle of wi...,"[implementation, constructive algorithms, data..."
4,Of course you have heard the famous task about...,"[greedy, dp, data structures, brute force, sor..."


In [9]:
print("Train DataFrame Columns:")
print(train_df.columns)
print("\nTest DataFrame Columns:")
print(test_df.columns)

print("\nTrain DataFrame Head:")
display(train_df.head())
print("\nTest DataFrame Head:")
display(test_df.head())

Train DataFrame Columns:
Index(['description', 'tags'], dtype='object')

Test DataFrame Columns:
Index(['description', 'tags'], dtype='object')

Train DataFrame Head:


,description,tags
0,After years of hard work scientists invented a...,"[greedy, constructive algorithms]"
1,Traveling around the world you noticed that ma...,[dp]
2,"Burenka is the crown princess of Buryatia, and...","[greedy, math, dp]"
3,Once Bob needed to find the second order stati...,[brute force]
4,Petya and Gena love playing table tennis. A si...,[binary search]



Test DataFrame Head:


,description,tags
0,The black king is standing on a chess field co...,[graphs]
1,"You are given an integer x of n digits a_1, a_...","[greedy, implementation, constructive algorithms]"
2,"You are organizing a boxing tournament, where ...","[greedy, dp, brute force]"
3,Sherlock met Moriarty for a final battle of wi...,"[implementation, constructive algorithms, data..."
4,Of course you have heard the famous task about...,"[greedy, dp, data structures, brute force, sor..."


In [10]:
AMT10 = [
    'implementation',
    'dp',
    'math',
    'greedy',
    'data structures',
    'brute force',
    'geometry',
    'constructive algorithms',
    'dfs and similar',
    'strings'
]

In [11]:
model_config = AutoConfig.from_pretrained("google/bigbird-roberta-base", max_position_embeddings=1024)
model_config

BigBirdConfig {
  "architectures": [
    "BigBirdForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "attention_type": "block_sparse",
  "block_size": 64,
  "bos_token_id": 1,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 1024,
  "model_type": "big_bird",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_random_blocks": 3,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "rescale_embeddings": false,
  "sep_token_id": 66,
  "transformers_version": "4.57.1",
  "type_vocab_size": 2,
  "use_bias": true,
  "use_cache": true,
  "vocab_size": 50358
}

In [12]:
config = {
    'seed' : 42,
    'tags' : AMT10,
    'batchSize' : 4,
    'lr' : 5e-6,
    'trainMaxLength' : 1024,
    'testMaxLength' : 1024,
    'numEpochs' : 50,
    'model' : AutoModel.from_config(model_config),
    'tokenizer' : RobertaTokenizer.from_pretrained('roberta-base'),
    'gradient_accumulation_steps' : 4,
    'max_grad_norm' : 1.0,
    'task' : 'tag', # rating, tag
    'lambda' : 10,
    'save' : True,
}

In [13]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed(config['seed'])

In [14]:
new_train_idx = []  # List to store new indices
selected_train_tags = []  # List to store selected tags

# Iterate through the DataFrame indices
for index in train_df.index:
    check = 0
    t = []  # List to store selected tags for this index

    # Iterate through the tags for the current index
    for tag in train_df.loc[index]['tags']: # Removed ast.literal_eval
        if tag in config['tags']:
            check = 1
            t.append(tag)

    # If at least one tag is in the desired tags list, append the index and selected tags
    if check == 1:
        selected_train_tags.append(t)
        new_train_idx.append(index)

print(len(new_train_idx))  # Print the length of the new index list

7385


In [15]:
new_test_idx = []  # List to store new indices (changed from valid_idx)
selected_test_tags = []  # List to store selected tags (changed from valid_tags)

# Iterate through the DataFrame indices
for index in test_df.index: # Changed from valid_df.index
    check = 0
    t = []  # List to store selected tags for this index

    # Iterate through the tags for the current index
    for tag in test_df.loc[index]['tags']: # Changed from valid_df.loc and removed ast.literal_eval
        if tag in config['tags']:
            check = 1
            t.append(tag)

    # If at least one tag is in the desired tags list, append the index and selected tags
    if check == 1:
        selected_test_tags.append(t)
        new_test_idx.append(index)

print(len(new_test_idx))  # Print the length of the new index list

813


In [16]:
train_df = train_df.loc[new_train_idx]
train_df['tags'] = selected_train_tags

test_df = test_df.loc[new_test_idx] # Changed from valid_df
test_df['tags'] = selected_test_tags # Changed from selected_valid_tags

In [17]:
train_df['rating'] = np.zeros(len(train_df)).astype(int)

# Split the training data into training and validation sets
# Using a fixed random_state for reproducibility
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=config['seed'])

X_train = train_data['description']
y_tags_train = train_data['tags']
y_ratings_train = train_data['rating']

X_val = val_data['description'] # This will be the new validation set
y_tags_val = val_data['tags']
y_ratings_val = val_data['rating']

# X_test and related y_tags_test, y_ratings_test will remain but won't be used in Trainer
# as the user requested to use validation data instead of testing data in the training.
X_test_final = test_df['description'] # Renaming to avoid confusion with validation
y_tags_test_final = test_df['tags']
y_ratings_test_final = np.zeros(len(test_df)).astype(int) # Dummy ratings for final test set

In [19]:
# Robust diagnostic: print which AMT10 tags are present/missing and counts
from collections import Counter
from itertools import chain
import ast
import pandas as pd

def extract_tag_list(value):
    # Turn a single cell value into a list of tag strings (safe)
    if isinstance(value, (list, tuple, set)):
        return list(value)
    if isinstance(value, str):
        v = value.strip()
        # try parse stringified list like "['dp','math']"
        if v.startswith('[') and v.endswith(']'):
            try:
                parsed = ast.literal_eval(v)
                if isinstance(parsed, (list, tuple, set)):
                    return list(parsed)
            except Exception:
                pass
        # otherwise treat the string as a single tag or comma-separated list
        if ',' in v and not v.startswith('['):
            return [p.strip() for p in v.split(',') if p.strip()]
        return [v]
    # fallback: coerce to string
    return [str(value)]

def normalize_list(series):
    out = []
    for val in series:
        tags = extract_tag_list(val)
        for t in tags:
            if t is None:
                continue
            out.append(str(t).strip().lower())
    return out

# Ensure the splits exist; prefer the explicit train_data if available
if 'train_data' in globals():
    train_series = train_data['tags']
elif 'train_df' in globals():
    train_series = train_df['tags']
else:
    raise NameError('train_df or train_data not defined. Run cells up to the train/test split first.')

val_series = val_data['tags'] if 'val_data' in globals() else pd.Series([], dtype=object)
test_series = test_df['tags'] if 'test_df' in globals() else pd.Series([], dtype=object)

train_tags_norm = normalize_list(train_series)
val_tags_norm = normalize_list(val_series)
test_tags_norm = normalize_list(test_series)

print('Unique tags in TRAIN:', len(set(train_tags_norm)))
print('Top TRAIN tag counts:', Counter(train_tags_norm).most_common(50))
print()
print('Unique tags in VAL:', len(set(val_tags_norm)))
print('Top VAL tag counts:', Counter(val_tags_norm).most_common(50))
print()
print('Unique tags in TEST (final):', len(set(test_tags_norm)))
print('Top TEST tag counts:', Counter(test_tags_norm).most_common(50))
print()
AMT10_norm = [t.strip().lower() for t in AMT10] if 'AMT10' in globals() else []
missing = [t for t in AMT10_norm if t not in set(train_tags_norm)]
print('AMT10 (normalized):', AMT10_norm)
print('AMT10 missing from TRAIN:', missing)
print()
print('Sample train tags rows (first 10):')
sample_src = train_data['tags'] if 'train_data' in globals() else train_df['tags']
for i, row in enumerate(sample_src.head(10)):
    print(i, row)

Unique tags in TRAIN: 7
Top TRAIN tag counts: [('greedy', 2137), ('math', 2093), ('implementation', 1770), ('dp', 1482), ('constructive algorithms', 1242), ('data structures', 1231), ('brute force', 1228)]

Unique tags in VAL: 7
Top VAL tag counts: [('greedy', 526), ('math', 503), ('implementation', 461), ('dp', 395), ('data structures', 320), ('constructive algorithms', 309), ('brute force', 295)]

Unique tags in TEST (final): 7
Top TEST tag counts: [('greedy', 291), ('implementation', 270), ('math', 265), ('dp', 212), ('constructive algorithms', 178), ('data structures', 173), ('brute force', 148)]

AMT10 (normalized): ['implementation', 'dp', 'math', 'greedy', 'data structures', 'brute force', 'geometry', 'constructive algorithms', 'dfs and similar', 'strings']
AMT10 missing from TRAIN: ['geometry', 'dfs and similar', 'strings']

Sample train tags rows (first 10):
0 ['math', 'dp']
1 ['implementation', 'dp']
2 ['greedy']
3 ['greedy']
4 ['math', 'brute force']
5 ['greedy', 'math', 'co

In [ ]:
# Create an instance of the MultiLabelBinarizer
tag_label_encoder = MultiLabelBinarizer()
rating_label_encoder = LabelEncoder()

# Fit the label encoder on the training labels and transform them
y_tags_train_encoded = tag_label_encoder.fit_transform(y_tags_train)
y_tags_val_encoded = tag_label_encoder.transform(y_tags_val) # Transform validation set
y_tags_test_final_encoded = tag_label_encoder.transform(y_tags_test_final) # Transform final test set

y_ratings_train_encoded = rating_label_encoder.fit_transform(y_ratings_train)
y_ratings_val_encoded = rating_label_encoder.transform(y_ratings_val) # Transform validation set
y_ratings_test_final_encoded = rating_label_encoder.transform(y_ratings_test_final) # Transform final test set

In [ ]:
# Define a class for multi-label classification head
class MultiLabelClassificationHead(nn.Module):
    def __init__(self, num_labels, hidden_size=768):
        super().__init__()
        self.fc1 = nn.Linear(hidden_size, hidden_size) # Fully connected layer
        self.fc2 = nn.Linear(hidden_size, hidden_size) # Fully connected layer
        self.fc3 = nn.Linear(hidden_size, num_labels) # Fully connected layer
        self.sigmoid = nn.Sigmoid() # Sigmoid activation function

    def forward(self, x):
        x = self.fc1(x) # Apply the fully connected layer
        x = self.fc2(x) # Apply the fully connected layer
        x = self.fc3(x) # Apply the fully connected layer
        x = self.sigmoid(x) # Apply the sigmoid activation
        return x

# Define a class for multi-class classification head
class MultiClassClassificationHead(nn.Module):
    def __init__(self, num_labels, hidden_size=768):
        super().__init__()
        self.fc = nn.Linear(hidden_size, num_labels)  # Fully connected layer

    def forward(self, x):
        x = self.fc(x)  # Apply the fully connected layer
        return x

# Define a classifier class
class classifier(nn.Module):
    def __init__(self, model, device, tags_num_classes, ratings_num_classes):
        super().__init__()
        self.tags_num_classes = tags_num_classes  # Number of classes for tags
        self.ratings_num_classes = ratings_num_classes  # Number of classes for ratings

        # Set the device (GPU or CPU)
        self.device = device

        # Initialize multi-label and multi-class classifiers
        self.tags_classifier = MultiLabelClassificationHead(num_labels=self.tags_num_classes).to(self.device)
        self.ratings_classifier = MultiClassClassificationHead(num_labels=self.ratings_num_classes).to(self.device)

        # Define loss functions for multi-label and multi-class classification
        self.BCE = nn.BCELoss().to(self.device)  # Binary Cross Entropy loss for multi-label classification
        self.CE = nn.CrossEntropyLoss().to(self.device)  # Cross Entropy loss for multi-class classification

        self.model = model
        self.lr = config['lr']
        self.task = config['task']

        if self.task == 'tag':
            self.parameters = [
                    {'params': self.model.parameters()},
                    {'params': self.tags_classifier.parameters()},
                ]
        elif self.task == 'rating':
            self.parameters = [
                    {'params': self.model.parameters()},
                    {'params': self.ratings_classifier.parameters()}
                ]

        # Initialize the optimizer
        self.optimizer = torch.optim.Adam(
            self.parameters,
            lr=self.lr
        )

    def forward(self, input_ids, attention_mask, tags_labels, ratings_labels):

        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output # Pooled output from the model

        if self.task == 'tag':
          output = self.tags_classifier(pooled_output) # Predict tags using the tags classifier
          loss = self.BCE(output, tags_labels) # Calculate the loss for tags
        elif self.task == 'rating':
          output = self.ratings_classifier(pooled_output) # Predict ratings using the ratings classifier
          loss = self.CE(output, ratings_labels) # Calculate the loss for ratings

        return loss, output

In [ ]:
def tokenizing(tokenizer, data, max_length):
    # Tokenize and encode the text input
    data = list(data.values)
    tokenized_data = tokenizer(data, padding=True, truncation=True, return_tensors='pt', max_length=max_length)

    return tokenized_data

In [ ]:
def convert_to_tensor(data, dtype):
    # Convert data to tensors
    tensor_data = torch.tensor(data, dtype=dtype)
    return tensor_data

In [ ]:
class Trainer():
    def __init__(self,
                 model,
                 tokenized_inputs_train,
                 tokenized_inputs_val,
                 tags_labels_train,
                 tags_labels_val,
                 ratings_labels_train,
                 ratings_labels_val
                ):
        # Set the device (GPU or CPU)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Store the input data and labels
        self.tokenized_inputs_train = tokenized_inputs_train
        self.tokenized_inputs_val = tokenized_inputs_val

        self.tags_labels_train = tags_labels_train
        self.tags_labels_val = tags_labels_val

        self.ratings_labels_train = ratings_labels_train
        self.ratings_labels_val = ratings_labels_val

        # Determine the number of classes for tags and ratings
        self.tags_num_classes = len(tags_labels_train[0])
        self.ratings_num_classes = len(np.unique(ratings_labels_train))

        # Move the model to the specified device
        self.model = model.to(self.device)

        # Define Classifier Instance
        self.classifier_instance = classifier(self.model, self.device, self.tags_num_classes, self.ratings_num_classes)

        # Retrieve configuration parameters
        self.batch_size = config['batchSize']
        self.num_epochs = config['numEpochs']

        self.accumulation_steps = config['gradient_accumulation_steps']
        self.max_grad_norm = config['max_grad_norm']

        self.tag_classes = tag_label_encoder.classes_

        self.save = config['save']

        # Initialize input data variables
        self.input_ids_train = self.tokenized_inputs_train['input_ids']
        self.attention_mask_train = self.tokenized_inputs_train['attention_mask']

        self.input_ids_val = tokenized_inputs_val['input_ids']
        self.attention_mask_val = tokenized_inputs_val['attention_mask']

        self.task = config['task']

    def train(self):
        input_ids_train = self.input_ids_train
        attention_mask_train = self.attention_mask_train
        tags_labels_train = self.tags_labels_train
        ratings_labels_train = self.ratings_labels_train

        input_ids_val = self.input_ids_val
        attention_mask_val = self.attention_mask_val
        tags_labels_val = self.tags_labels_val
        ratings_labels_val = self.ratings_labels_val

        # Set the optimizer and learning rate
        optimizer = self.classifier_instance.optimizer
        parameters = self.classifier_instance.parameters

        # Set the batch size
        batch_size = self.batch_size

        # Create a DataLoader for batching the data
        train_dataset = TensorDataset(input_ids_train, attention_mask_train, tags_labels_train, ratings_labels_train)
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=8, pin_memory=True)

        valid_dataset = TensorDataset(input_ids_val, attention_mask_val, tags_labels_val, ratings_labels_val) # Using validation data
        valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

        # Set the number of training epochs#
        num_epochs = self.num_epochs
        device = self.device

        model = self.model
        classifier_instance = self.classifier_instance

        # Training loop
        min_loss = 999999
        rating_f1_s = 0
        total_f1_s = 0
        count = 0

        #epochs

        max_total_f1_macro_score_epochs = 0

        max_tag_acc_epochs = 0
        max_tag_f1_macro_epochs = 0
        max_tag_f1_micro_epochs = 0
        max_tag_f1_weighted_epochs= 0
        max_tag_f1_samples_epochs= 0
        max_tag_roc_auc_score_epochs = 0

        max_rating_acc_epochs = 0
        max_rating_f1_macro = 0
        max_rating_f1_micro = 0
        max_rating_f1_weighted = 0

        #score
        max_total_f1_macro_score = 0

        max_tag_acc = 0
        max_tag_f1_macro = 0
        max_tag_f1_micro = 0
        max_tag_f1_weighted = 0
        max_tag_f1_samples = 0
        max_tag_roc_auc_score = 0

        max_rating_acc = 0
        max_rating_f1_macro = 0
        max_rating_f1_micro = 0
        max_rating_f1_weighted = 0

        thresholds = [0.001] + [i * 0.01 for i in range(1, 101)]

        for epoch in range(num_epochs):
            # set early stopping
            #if count > 8:
            #    break
            train_loss = 0.0
            valid_loss = 0.0

            tags_true = []
            tags_pred = defaultdict(list)
            tags_pred_proba = []

            ratings_true = []
            ratings_pred = []

            count += 1

            # Training
            classifier_instance.train()
            # Zero the gradients
            optimizer.zero_grad()
            for batch in tqdm(train_dataloader):
                # Unpack the batch
                input_ids, attention_mask, tags_labels, ratings_labels = batch

                # Move the inputs and labels to the chosen device
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                tags_labels = tags_labels.to(device)
                ratings_labels = ratings_labels.to(device)

                # Forward pass
                loss, _ = classifier_instance(input_ids, attention_mask, tags_labels, ratings_labels)
                loss /= self.accumulation_steps

                # Backward pass and optimization
                loss.backward()

                if epoch % self.accumulation_steps ==  0 or epoch == batch_size - 1 or self.accumulation_steps == 0:
                    if self.max_grad_norm > 0:

                        if self.task == 'tag':
                            torch.nn.utils.clip_grad_norm_(chain(
                                model.parameters(),
                                classifier_instance.tags_classifier.parameters()
                            ), self.max_grad_norm)
                        elif self.task == 'rating':
                            torch.nn.utils.clip_grad_norm_(chain(
                                model.parameters(),
                                classifier_instance.ratings_classifier.parameters()
                            ), self.max_grad_norm)
                    optimizer.step()
                    optimizer.zero_grad()

                train_loss += loss.item()

            # Validation
            #model.eval()
            classifier_instance.eval()
            with torch.no_grad():
                for batch in tqdm(valid_dataloader):
                    # Unpack the batch
                    input_ids, attention_mask, tags_labels, ratings_labels = batch

                    # Move the inputs and labels to the chosen device
                    input_ids = input_ids.to(device)
                    attention_mask = attention_mask.to(device)
                    tags_labels = tags_labels.to(device)
                    ratings_labels = ratings_labels.to(device)

                    # Forward pass
                    loss, output = classifier_instance(input_ids, attention_mask, tags_labels, ratings_labels)

                    valid_loss += loss.item()

                    if self.task == 'tag':
                        tags_output = output

                        # tags
                        tags_pred_proba.extend(tags_output.detach().cpu().clone().tolist())

                        # Extract indices where the value is above the threshold.
                        for threshold in thresholds:
                            tags_pred[threshold].extend([(row >= threshold).nonzero().flatten().tolist() for row in tags_output.detach().cpu().clone()])

                        tags_true.extend([torch.nonzero(row).flatten().tolist() for row in tags_labels.detach().cpu().clone()])
                    elif self.task == 'rating':

                        ratings_output = output

                        ratings_pred.extend(torch.argmax(ratings_output, dim=1).detach().cpu().clone())
                        ratings_true.extend(ratings_labels.detach().cpu().clone())


            # Calculate average loss
            train_loss /= len(train_dataset)
            valid_loss /= len(valid_dataset)


            if epoch % self.accumulation_steps ==  0 or epoch == batch_size - 1 or self.accumulation_steps == 0:

                # Print the loss, F1 score, precision, and recall for monitoring
                print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}")

                tag_true = []
                #tag_pred = []

                if self.task == 'rating':

                    rating_true = [tensor.detach().cpu().clone().item() for tensor in ratings_true]
                    rating_pred = [tensor.detach().cpu().clone().item() for tensor in ratings_pred]

                    rating_k = defaultdict(list)

                    for k in [0, 1, 2]:
                        revise_rating_pred = []

                        for i in range(len(rating_pred)):
                            if abs(rating_true[i] - rating_pred[i]) <= k:
                                revise_rating_pred.append(rating_true[i])
                            else:
                                revise_rating_pred.append(rating_pred[i])
                        rating_k[k] = revise_rating_pred

                    rating_pred = rating_k[1]

                    rat_t = Counter(rating_true)
                    rat_p = Counter(rating_pred)

                    epoch_max_rating_acc = 0
                    epoch_max_rating_f1_macro = 0
                    epoch_max_rating_f1_micro = 0
                    epoch_max_rating_f1_weighted = 0

                    # rating

                    for k in [0, 1, 2]:
                        rating_pred = rating_k[k]

                        rating_acc = accuracy_score(rating_true, rating_pred)
                        rating_f1_macro = f1_score(rating_true, rating_pred, average='macro', zero_division=0)
                        rating_f1_micro = f1_score(rating_true, rating_pred, average='micro', zero_division=0)
                        rating_f1_weighted = f1_score(rating_true, rating_pred, average='weighted', zero_division=0)

                        if k == 1:
                            epoch_max_rating_acc = max(epoch_max_rating_acc, rating_acc)
                            epoch_max_rating_f1_macro = max(epoch_max_rating_f1_macro, rating_f1_macro)
                            epoch_max_rating_f1_micro = max(epoch_max_rating_f1_micro, rating_f1_micro)
                            epoch_max_rating_f1_weighted = max(epoch_max_rating_f1_weighted, rating_f1_weighted)

                        #rating
                        print(f"rating acc Max Score in this epoch at {k}:", rating_acc)
                        print(f"rating valid Max F1 Score(macro) per class in this epoch at {k}:", rating_f1_macro)
                        print(f"rating valid Max F1 Score(micro) per class in this epoch at {k}:", rating_f1_micro)
                        print(f"rating valid Max F1 Score(weighted) per class in this epoch at {k}:", rating_f1_weighted)
                        print()

                    #rating
                    print("rating acc Max Score in this epoch:", epoch_max_rating_acc)
                    print("rating valid Max F1 Score(macro) per class in this epoch:", epoch_max_rating_f1_macro)
                    print("rating valid Max F1 Score(micro) per class in this epoch:", epoch_max_rating_f1_micro)
                    print("rating valid Max F1 Score(weighted) per class in this epoch:", epoch_max_rating_f1_weighted)
                    print()
                    print('rating_true : ', sorted(rat_t.items(), key=lambda x: x[0]))
                    print('rating_pred : ', sorted(rat_p.items(), key=lambda x: x[0]))
                    print()


                    #rating
                    print(f"rating acc Max Score: {max_rating_acc} at {max_rating_acc_epochs}epochs")
                    print(f"rating valid Max F1 Score(macro) per class: {max_rating_f1_macro} at {max_rating_f1_macro_epochs}epochs")
                    print(f"rating valid Max F1 Score(micro) per class: {max_rating_f1_micro} at {max_rating_f1_micro_epochs}epochs")
                    print(f"rating valid Max F1 Score(weighted) per class: {max_rating_f1_weighted} at {max_rating_f1_weighted_epochs}epochs")
                    print()

                    # rating

                    if epoch_max_rating_acc > max_rating_acc:
                        max_rating_acc_epochs = epoch
                        max_rating_acc = max(epoch_max_rating_acc, max_rating_acc)

                    if epoch_max_rating_f1_macro > max_rating_f1_macro:
                        max_rating_f1_macro_epochs = epoch
                        max_rating_f1_macro = max(epoch_max_rating_f1_macro, max_rating_f1_macro)

                        now = datetime.now()
                        task = config['task']
                        if self.save:
                            self.save_checkpoint(task, model, epoch)
                        count = 0
                        print('Best Model Saved !')
                        print()

                    if epoch_max_rating_f1_micro > max_rating_f1_micro:
                        max_rating_f1_micro_epochs = epoch
                        max_rating_f1_micro = max(epoch_max_rating_f1_micro, max_rating_f1_micro)

                    if epoch_max_rating_f1_weighted > max_rating_f1_weighted:
                        max_rating_f1_weighted_epochs = epoch
                        max_rating_f1_weighted = max(epoch_max_rating_f1_weighted, max_rating_f1_weighted)

                elif self.task == 'tag':

                    for index_list in tags_true:
                        result_true = [0] * self.tags_num_classes  # Create a list of length num_classes.
                        for index in index_list:
                            result_true[index] = 1  # Fill the corresponding index with 1.

                        tag_true.append(result_true)


                    epoch_max_tag_acc = 0
                    epoch_max_tag_f1_macro = 0
                    epoch_max_tag_f1_micro = 0
                    epoch_max_tag_f1_weighted = 0
                    epoch_max_tag_f1_samples = 0

                    epoch_max_tag_roc_auc_score = roc_auc_score(tag_true, tags_pred_proba)
                    tag_true = np.array(tag_true)
                    tags_pred_proba = np.array(tags_pred_proba)

                    for threshold in thresholds:
                        tag_pred = []
                        for index_list in tags_pred[threshold]:
                            result_pred = [0] * self.tags_num_classes  # Create a list of length num_classes.
                            for index in index_list:
                                result_pred[index] = 1 # Fill the corresponding index with 1.

                            tag_pred.append(result_pred)

                        # tag

                        tag_acc = accuracy_score(tag_true, tag_pred)
                        tag_f1_macro = f1_score(tag_true, tag_pred, average='macro', zero_division=0)
                        tag_f1_micro = f1_score(tag_true, tag_pred, average='micro', zero_division=0)
                        tag_f1_weighted = f1_score(tag_true, tag_pred, average='weighted', zero_division=0)
                        tag_f1_samples = f1_score(tag_true, tag_pred, average='samples', zero_division=0)

                        epoch_max_tag_acc = max(epoch_max_tag_acc, tag_acc)
                        epoch_max_tag_f1_macro = max(epoch_max_tag_f1_macro, tag_f1_macro)
                        epoch_max_tag_f1_micro = max(epoch_max_tag_f1_micro, tag_f1_micro)
                        epoch_max_tag_f1_weighted = max(epoch_max_tag_f1_weighted, tag_f1_weighted)
                        epoch_max_tag_f1_samples = max(epoch_max_tag_f1_samples, tag_f1_samples)

                    #tag
                    print("tag acc Max Score in this epoch:", epoch_max_tag_acc)
                    print("tag valid Max F1 Score(macro) per class in this epoch:", epoch_max_tag_f1_macro)
                    print("tag valid Max F1 Score(micro) per class in this epoch:", epoch_max_tag_f1_micro)
                    print("tag valid Max F1 Score(weighted) per class in this epoch:", epoch_max_tag_f1_weighted)
                    print("tag valid Max F1 Score(samples) per class in this epoch:", epoch_max_tag_f1_samples)
                    print()
                    print("tag valid Max roc_auc_score avg in this epoch:", epoch_max_tag_roc_auc_score)

                    for num_classes in range(self.tags_num_classes):
                        score = roc_auc_score(tag_true[:, num_classes], tags_pred_proba[:, num_classes])
                        print(f"{self.tag_classes[num_classes]} : {score}")
                    print()

                    #tag
                    print(f"tag acc Max Score: {max_tag_acc} at {max_tag_acc_epochs}epochs")
                    print(f"tag valid Max F1 Score(macro) per class: {max_tag_f1_macro} at {max_tag_f1_macro_epochs}epochs")
                    print(f"tag valid Max F1 Score(micro) per class: {max_tag_f1_micro} at {max_tag_f1_micro_epochs}epochs")
                    print(f"tag valid Max F1 Score(weighted) per class: {max_tag_f1_weighted} at {max_tag_f1_weighted_epochs}epochs")
                    print(f"tag valid Max F1 Score(samples) per class: {max_tag_f1_samples} at {max_tag_f1_samples_epochs}epochs")
                    print(f"tag valid Max roc_auc_score: {max_tag_roc_auc_score} at {max_tag_roc_auc_score_epochs}epochs")
                    print()

                    # tag

                    if epoch_max_tag_acc > max_tag_acc:
                        max_tag_acc_epochs = epoch
                        max_tag_acc = max(epoch_max_tag_acc, max_tag_acc)

                    if epoch_max_tag_f1_macro > max_tag_f1_macro:
                        max_tag_f1_macro_epochs = epoch
                        max_tag_f1_macro = max(epoch_max_tag_f1_macro, max_tag_f1_macro)

                        now = datetime.now()
                        task = config['task']
                        if self.save:
                            self.save_checkpoint(task, model, epoch)
                        count = 0
                        print('Best Model Saved !')
                        print()

                    if epoch_max_tag_f1_micro > max_tag_f1_micro:
                        max_tag_f1_micro_epochs = epoch
                        max_tag_f1_micro = max(epoch_max_tag_f1_micro, max_tag_f1_micro)

                    if epoch_max_tag_f1_weighted > max_tag_f1_weighted:
                        max_tag_f1_weighted_epochs = epoch
                        max_tag_f1_weighted = max(epoch_max_tag_f1_weighted, max_tag_f1_weighted)

                    if epoch_max_tag_f1_samples > max_tag_f1_samples:
                        max_tag_f1_samples_epochs = epoch
                        max_tag_f1_samples = max(epoch_max_tag_f1_samples, max_tag_f1_samples)

                    if epoch_max_tag_roc_auc_score > max_tag_roc_auc_score:
                        max_tag_roc_auc_score_epochs = epoch
                        max_tag_roc_auc_score = max(epoch_max_tag_roc_auc_score, max_tag_roc_auc_score)

                print('----------------------------------------------------------------------------')
                print()

    def save_checkpoint(self, task, model, epoch, max_checkpoints=5):
        now = datetime.now()
        today = now.strftime('%Y-%m-%d')
        checkpoint_filename = f"{now.strftime('%Y-%m-%d')}_{epoch + 1}"
        checkpoint_path = os.path.join(f"./models/{task}/{today}", checkpoint_filename)

        # If the directory does not exist, create it.
        if not os.path.exists(checkpoint_path):
            os.makedirs(checkpoint_path)

        # Save the model state_dict
        torch.save(self.classifier_instance.state_dict(), os.path.join(checkpoint_path, f"model.pt"))
        checkpoint_files = sorted(os.listdir(f"./models/{task}/{today}"))
        # Delete oldest checkpoint if there are too many
        while len(checkpoint_files) > max_checkpoints + 1:
            checkpoint_files = sorted(os.listdir(f"./models/{task}/{today}"))
            oldest_checkpoint = os.path.join(f"./models/{task}/{today}", checkpoint_files[0])
            #os.remove(oldest_checkpoint)
            if os.path.exists(oldest_checkpoint) and os.path.isdir(oldest_checkpoint):
                # Check if the directory is empty, and if not, use shutil.rmtree() to recursively delete it.
                try:
                    shutil.rmtree(oldest_checkpoint)
                except Exception as e:
                    print(f"Error while deleting directory: {e}")

In [ ]:
tokenizer = config['tokenizer']
model = config['model']

In [ ]:
tokenized_inputs_train = tokenizing(tokenizer, X_train, config['trainMaxLength'])
tokenized_inputs_val = tokenizing(tokenizer, X_val, config['testMaxLength']) # New validation tokenization
tokenized_inputs_test_final = tokenizing(tokenizer, X_test_final, config['testMaxLength']) # Final test tokenization

In [ ]:
tags_labels_train = convert_to_tensor(y_tags_train_encoded, dtype=torch.float)
tags_labels_val = convert_to_tensor(y_tags_val_encoded, dtype=torch.float) # New validation tensor
ratings_labels_train = convert_to_tensor(y_ratings_train_encoded, dtype=torch.long)
ratings_labels_val = convert_to_tensor(y_ratings_val_encoded, dtype=torch.long) # New validation tensor

# Keep test_final for later, but not pass to Trainer for validation
tags_labels_test_final = convert_to_tensor(y_tags_test_final_encoded, dtype=torch.float)
ratings_labels_test_final = convert_to_tensor(y_ratings_test_final_encoded, dtype=torch.long)

In [ ]:
trainer = Trainer(model,
                 tokenized_inputs_train,
                 tokenized_inputs_val, # Use validation set here
                 tags_labels_train,
                 tags_labels_val, # Use validation set here
                 ratings_labels_train,
                 ratings_labels_val # Use validation set here
                 )

In [ ]:
trainer.train()

  0%|          | 0/1477 [00:00<?, ?it/s]/Users/klarakjomefischer/Downloads/Carleton/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
Input ids are automatically padded from 1011 to 1024 to be a multiple of `config.block_size`: 64
 12%|█▏        | 174/1477 [22:15<2:55:45,  8.09s/it]libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x10d705ee0>
Traceback (most recent call last):
  File "/Users/klarakjomefischer/Downloads/Carleton/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/Users/klarakjomefischer/Downloads/Carleton/.venv/lib/python3.12/si

KeyboardInterrupt: 

# Inference

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

In [ ]:
state = torch.load('./model.pt')

In [ ]:
model_state_dict = {}
tag_state_dict = {}
rating_state_dict = {}

In [ ]:
for k, v in state.items():
    if "model." in k:
        name = k[6:]
        model_state_dict[name] = v
    if "tags_classifier." in k:
        name = k[len("tags_classifier."):]
        tag_state_dict[name] = v
    if "ratings_classifier." in k:
        name = k[len("ratings_classifier."):]
        rating_state_dict[name] = v

In [ ]:
model = config['model']
tag_head = MultiLabelClassificationHead(10)
rating_head = MultiClassClassificationHead(28)

In [ ]:
model.load_state_dict(model_state_dict)
tag_head.load_state_dict(tag_state_dict)
rating_head.load_state_dict(rating_state_dict)
print('fin')

In [ ]:
# Set the device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the chosen device
model.to(device)
tag_head.to(device)
rating_head.to(device)
print('device : ', device)

In [ ]:
valid_dataset = TensorDataset(tokenized_inputs_test_final['input_ids'], tokenized_inputs_test_final['attention_mask'], tags_labels_test_final, ratings_labels_test_final)
valid_dataloader = DataLoader(valid_dataset, batch_size=config['batchSize'], shuffle=False, num_workers=8, pin_memory=True)

In [ ]:
model.eval()
tag_head.eval()
rating_head.eval()
with torch.no_grad():

    thresholds = [0.001] + [i * 0.01 for i in range(1, 101)]
    tags_true = []
    tags_pred = defaultdict(list)
    tags_pred_proba = []

    ratings_true = []
    ratings_pred = []
    for batch in tqdm(valid_dataloader):
        ## Unpack the batch
        input_ids, attention_mask, tags_labels, ratings_labels = batch

        # Move the inputs and labels to the chosen device
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        tags_labels = tags_labels.to(device)
        ratings_labels = ratings_labels.to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output

        tags_output = tag_head(pooled_output)
        ratings_output = rating_head(pooled_output)

        # tags
        tags_true.extend([torch.nonzero(row).flatten().tolist() for row in tags_labels.detach().cpu().clone()])
        tags_pred_proba.extend(tags_output.detach().cpu().clone().tolist())

        ratings_pred.extend(torch.argmax(ratings_output, dim=1).detach().cpu().clone())
        ratings_true.extend(ratings_labels.detach().cpu().clone())

        # Extract indices with values greater than or equal to the threshold.
        for threshold in thresholds:
            tags_pred[threshold].extend([(row >= threshold).nonzero().flatten().tolist() for row in tags_output.detach().cpu().clone()])

    rating_true = [tensor.detach().cpu().clone().item() for tensor in ratings_true]
    rating_pred = [tensor.detach().cpu().clone().item() for tensor in ratings_pred]

    revise_rating_pred = []

    for i in range(len(rating_pred)):
        if abs(rating_true[i] - rating_pred[i]) <= 1:
            revise_rating_pred.append(rating_true[i])
        else:
            revise_rating_pred.append(rating_pred[i])

    rating_pred = revise_rating_pred

    tag_true = []

    for index_list in tags_true:
        result_true = [0] * 10  # Create a list of length num_classes.
        for index in index_list:
            result_true[index] = 1  # Fill the corresponding index with 1.

        tag_true.append(result_true)

    tag_true = np.array(tag_true)
    tags_pred_proba = np.array(tags_pred_proba)

    thr = 0
    max_f1_score = 0

    for threshold in thresholds:
        tag_pred = []
        for index_list in tags_pred[threshold]:
            result_pred = [0] * 10 # Create a list of length num_classes.
            for index in index_list:
                result_pred[index] = 1  # Fill the corresponding index with 1.

            tag_pred.append(result_pred)

        f1 = f1_score(tag_true, tag_pred, average='macro', zero_division=0)
        if max_f1_score < f1:
            thr = threshold
            max_f1_score = f1

    fpr = dict()
    tpr = dict()

    # Plot ROC curve for each classifier
    plt.figure()
    for num_classes in range(10):
        tt, tp = tag_true[:, num_classes], tags_pred_proba[:, num_classes]

        score = roc_auc_score(tt, tp)
        tag = tag_label_encoder.classes_[num_classes]
        print(f"{tag} : {score}")
        fpr[num_classes], tpr[num_classes], _ = roc_curve(tt, tp)
        plt.plot(fpr[num_classes], tpr[num_classes], label=f'{tag}(area={score:.2f})')
    print()

    print("tag_roc_auc_score : ", roc_auc_score(tag_true, tags_pred_proba))
    print("f1_score : ", max_f1_score)
    print("threshold : ", thr)

    rating_acc = accuracy_score(rating_true, rating_pred)
    print(f"rating_acc : {rating_acc}")

    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Tag Prediction')
    plt.legend(loc="lower right")
    plt.show()